In [32]:
import pandas as pd
import numpy as np
#载入数据
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data['origin'] = 'train'
test_data['origin'] = 'test'
data = pd.concat([train_data, test_data], ignore_index=True, sort=False)
#重命名列名
data.rename(columns = {'fecha_dato':'date','ncodpers':'user_id','ind_empleado':'employee_type',
                        'pais_residencia':'user_country','sexo':'sex','fecha_alta':'register_date',
                        'ind_nuevo':'new_user_type','antiguedad':'user_seniority','indrel':'user_type',
                        'ult_fec_cli_1t':'last_date_as_primary_user','indrel_1mes':'user_type_at_begin',
                        'tiprel_1mes':'user_relation_at_begin','indresi':'live_country_type','indext':'born_country_type',
                        'conyuemp':'employee_spouse_type','canal_entrada':'user_join_channel','indfall':'alive_type',
                        'tipodom':'address_type','cod_prov':'province_code','nomprov':'province_name',
                        'ind_actividad_cliente':'activity_type','renta':'user_income','segmento':'user_segmentation',
                        'ind_ahor_fin_ult1':'item_1','ind_aval_fin_ult1':'item_2','ind_cco_fin_ult1':'item_3',
                        'ind_cder_fin_ult1':'item_4','ind_cno_fin_ult1':'item_5','ind_ctju_fin_ult1':'item_6',
                        'ind_ctma_fin_ult1':'item_7','ind_ctop_fin_ult1':'item_8','ind_ctpp_fin_ult1':'item_9',
                        'ind_deco_fin_ult1':'item_10','ind_deme_fin_ult1':'item_11','ind_dela_fin_ult1':'item_12',
                        'ind_ecue_fin_ult1':'item_13','ind_fond_fin_ult1':'item_14','ind_hip_fin_ult1':'item_15',
                        'ind_plan_fin_ult1':'item_16','ind_pres_fin_ult1':'item_17','ind_reca_fin_ult1':'item_18',
                        'ind_tjcr_fin_ult1':'item_19','ind_valo_fin_ult1':'item_20','ind_viv_fin_ult1':'item_21',
                        'ind_nomina_ult1':'item_22','ind_nom_pens_ult1':'item_23','ind_recibo_ult1':'item_24'},inplace = True)
#去除一些相关性不太大的特征列
data = data.dropna(axis=0,subset = ['register_date'])
data = data.drop('employee_spouse_type',axis = 1)
data = data.drop('last_date_as_primary_user',axis = 1)
data = data.drop('province_name',axis = 1)
data = data.drop('address_type',axis = 1)
data = data.drop('employee_type',axis = 1)
#对特征列的空值进行补全
data['user_type'] = data['user_type'].replace(np.nan, 2)
data['province_code'] = data['province_code'].replace(np.nan, 99)
data['activity_type'] = data['activity_type'].replace(np.nan, 2)
data['user_country'] = data['user_country'].replace(np.nan, 'unknown')
data['sex'] = data['sex'].replace(np.nan,'unknown')
data['user_seniority'] = data['user_seniority'].replace(np.nan, 0)
data['user_seniority'] = data['user_seniority'].replace('     NA', 0)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('1.0', 1)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('1', 1)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('3.0', 3)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('3', 3)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('2.0', 2)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('2', 2)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('4.0', 4)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('4', 4)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('P', 5)
data['user_type_at_begin'] = data['user_type_at_begin'].replace(np.nan, 0)
data['user_relation_at_begin'] = data['user_relation_at_begin'].replace(np.nan, 'unknown')
data['live_country_type'] = data['live_country_type'].replace(np.nan, 'unknown')
data['born_country_type'] = data['born_country_type'].replace(np.nan, 'unknown')
data['user_join_channel'] = data['user_join_channel'].replace(np.nan, 'unknown')
data = data.drop('alive_type',axis = 1)
data['user_segmentation'] = data['user_segmentation'].replace(np.nan, 'unknown')
#增加用户使用时长的特征
data['date']=pd.to_datetime(data['date'],format='%Y-%m-%d')
data['register_date']=pd.to_datetime(data['register_date'],format='%Y-%m-%d')
data['use_time'] = (data['date'] - data['register_date']).dt.days
data = data.drop('register_date',axis =1)


C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
#提取用户信息特征准备通过xgboost对用户收入进行补全
user_data = data[['user_id','date','user_country','sex','age',
                'user_seniority','user_type','live_country_type','born_country_type',
                'user_join_channel','province_code','activity_type','user_income',
                'user_segmentation','use_time']]
user_data = user_data.sort_values(by=['user_id','date'])
user_data.drop_duplicates(subset = ['user_id'],keep='last',inplace=True)
user_data = user_data.drop('date',axis = 1)
user_data = user_data.reset_index(drop = True)


In [34]:
print(user_data)

       user_id user_country sex age user_seniority  user_type  \
0        16074           ES   V  47             83        1.0   
1        16095           ES   V  74            255        1.0   
2        16117           ES   V  57            256        1.0   
3        16181           ES   H  77            242        1.0   
4        16308           ES   V  47            248        1.0   
...        ...          ...  ..  ..            ...        ...   
39692  1548081           ES   H  44              1        1.0   
39693  1548083           ES   V  22              0        1.0   
39694  1548094           ES   H  56              0        1.0   
39695  1548100           ES   H  54              1        1.0   
39696  1548191           ES   H  70              0        1.0   

      live_country_type born_country_type user_join_channel  province_code  \
0                     S                 N               KFA           28.0   
1                     S                 N               KAT    

In [35]:
#对用户信息特征列进行labelencoder
user_data1 = user_data[['user_id','sex','age','province_code','user_segmentation','user_income']]
from sklearn.preprocessing import LabelEncoder
lbe_sex = LabelEncoder()
user_data1['sex'] = lbe_sex.fit_transform(user_data1['sex'])
lbe_province_code = LabelEncoder()
user_data1['province_code'] = lbe_province_code.fit_transform(user_data1['province_code'])
lbe_user_segmentation = LabelEncoder()
user_data1['user_segmentation'] = lbe_user_segmentation.fit_transform(user_data1['user_segmentation'])
user_data1['age'] = user_data1['age'].astype('int16')
print(user_data1)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1     53478.18
1        16095    1   74             27                  0          NaN
2        16117    1   57             27                  1    465589.68
3        16181    0   77             27                  1    134978.88
4        16308    1   47             27                  1    175988.07
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1          NaN
39693  1548083    1   22              6                  3          NaN
39694  1548094    0   56              2                  3          NaN
39695  1548100    0   54             28                  1          NaN
39696  1548191    0   70             27                  3          NaN

[39697 rows x 6 columns]


C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [36]:
#将存在收入数据的用户信息作为训练集，不存在收入数据的用户信息作为预测集
user_data_train = user_data1.dropna()
user_data_train['user_income'] = user_data_train['user_income'].astype('int64')
user_data_predict = user_data1[user_data1.user_income.isnull()]

C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
print(user_data_train)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
36184  1454440    0   24             27                  2        85797
36185  1454443    1   21             26                  2        96142
36186  1454499    1   20             27                  1       194682
36187  1454504    0   20             14                  2        54385
36188  1454511    1   28             14                  2       202631

[29850 rows x 6 columns]


In [38]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation  user_income
1        16095    1   74             27                  0          NaN
32       17453    0   75             27                  1          NaN
41       17701    1   53             27                  1          NaN
53       18024    1   60             27                  1          NaN
58       18288    1   70             38                  1          NaN
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1          NaN
39693  1548083    1   22              6                  3          NaN
39694  1548094    0   56              2                  3          NaN
39695  1548100    0   54             28                  1          NaN
39696  1548191    0   70             27                  3          NaN

[9847 rows x 6 columns]


In [39]:
user_data_predict = user_data_predict.drop('user_income',axis =1)

In [40]:
#将训练集中收入偏差均值3个标准差以外的收入数据进行剔除
user_income_mean = user_data_train['user_income'].mean()
print(user_income_mean)
user_income_std = user_data_train['user_income'].std()
print(user_income_std)
user_income_max = int(user_income_mean + 3 * user_income_std)
print(user_income_max)

136586.759798995
320021.55237731023
1096651


In [41]:
user_data_train1 = user_data_train[user_data_train.user_income < user_income_max]
print(user_data_train1)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
36184  1454440    0   24             27                  2        85797
36185  1454443    1   21             26                  2        96142
36186  1454499    1   20             27                  1       194682
36187  1454504    0   20             14                  2        54385
36188  1454511    1   28             14                  2       202631

[29786 rows x 6 columns]


In [42]:
#进行xgboost预测
import xgboost as xgb
from sklearn.model_selection import train_test_split

param = {'boosting_type':'gbdt',
                         'objective' : 'reg:linear', 
                         'eval_metric' : 'rmse',
                         'eta' : 0.01,
                         'max_depth' : 10,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         #'alpha': 0.6,
                         'lambda': 2,
        }
user_data_predict1 = xgb.DMatrix(user_data_predict[['sex','age','province_code','user_segmentation']])
#for name in col_name:
X_train, X_valid, y_train, y_valid = train_test_split(user_data_train1.drop(['user_income','user_id'],axis=1), user_data_train1['user_income']
                                                      , test_size=0.2, random_state=2020)

train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)

predict = model.predict(user_data_predict1)


[17:50:22] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:162567	valid-rmse:163332
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 200 rounds.
[25]	train-rmse:139612	valid-rmse:140551
[50]	train-rmse:123575	valid-rmse:124694
[75]	train-rmse:112720	valid-rmse:114050
[100]	train-rmse:105285	valid-rmse:106822
[125]	train-rmse:100349	valid-rmse:102180
[150]	train-rmse:97134.1	valid-rmse:99186
[175]	train-rmse:95033.1	valid-rmse:97262.8
[200]	train-rmse:93670.4	valid-rmse:96092.8
[225]	train-rmse:92774.1	valid-rmse:95357.9
[250]	train-rmse:92162.1	valid-rmse:94877.5
[275]	train-rmse:91733.3	valid-rmse:94601.3
[300]	train-rmse:91434.2	valid-rmse:94437
[325]	train-rmse:91202.7	valid-rmse:94351.1
[350]	train-rmse:91029.2	valid-rmse:94314
[375]	train-rmse:90872.2	valid-rmse:94303.4
[400]	tra

In [43]:
user_data_predict['user_income'] = predict

In [44]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation    user_income
1        16095    1   74             27                  0  221646.671875
32       17453    0   75             27                  1  155265.656250
41       17701    1   53             27                  1  165207.000000
53       18024    1   60             27                  1  166919.312500
58       18288    1   70             38                  1  135890.796875
...        ...  ...  ...            ...                ...            ...
39692  1548081    0   44             20                  1   81214.476562
39693  1548083    1   22              6                  3  156591.078125
39694  1548094    0   56              2                  3   77598.093750
39695  1548100    0   54             28                  1   95437.445312
39696  1548191    0   70             27                  3   95293.593750

[9847 rows x 6 columns]


In [45]:
user_data_predict['user_income'] = user_data_predict['user_income'].astype('int64')

In [46]:
print(user_data_train)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
36184  1454440    0   24             27                  2        85797
36185  1454443    1   21             26                  2        96142
36186  1454499    1   20             27                  1       194682
36187  1454504    0   20             14                  2        54385
36188  1454511    1   28             14                  2       202631

[29850 rows x 6 columns]


In [47]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation  user_income
1        16095    1   74             27                  0       221646
32       17453    0   75             27                  1       155265
41       17701    1   53             27                  1       165207
53       18024    1   60             27                  1       166919
58       18288    1   70             38                  1       135890
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1        81214
39693  1548083    1   22              6                  3       156591
39694  1548094    0   56              2                  3        77598
39695  1548100    0   54             28                  1        95437
39696  1548191    0   70             27                  3        95293

[9847 rows x 6 columns]


In [48]:
user_data_all = pd.concat([user_data_train,user_data_predict],ignore_index = True)

In [49]:
print(user_data_all)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
1        16117    1   57             27                  1       465589
2        16181    0   77             27                  1       134978
3        16308    1   47             27                  1       175988
4        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1        81214
39693  1548083    1   22              6                  3       156591
39694  1548094    0   56              2                  3        77598
39695  1548100    0   54             28                  1        95437
39696  1548191    0   70             27                  3        95293

[39697 rows x 6 columns]


In [50]:
print(data)

             date  user_id user_country sex  age  new_user_type  \
0      2015-01-28  1050676           ES   V   31            0.0   
1      2015-01-28  1050662           ES   V   37            0.0   
2      2015-01-28  1050652           ES   H   31            0.0   
3      2015-01-28  1050489           ES   H   23            0.0   
4      2015-01-28  1050490           ES   V   23            0.0   
...           ...      ...          ...  ..  ...            ...   
562927 2016-05-28  1166847           ES   V   23            0.0   
562928 2016-05-28  1166605           ES   V   27            0.0   
562929 2016-05-28  1166829           ES   V   23            0.0   
562930 2016-05-28  1166827           ES   H   23            0.0   
562931 2016-05-28  1166781           ES   H   24            0.0   

       user_seniority  user_type  user_type_at_begin user_relation_at_begin  \
0                  35        1.0                 1.0                      A   
1                  35        1.0     

In [52]:
user_data_all = user_data_all.drop(['sex','age','province_code','user_segmentation'],axis =1)
print(user_data_all)

       user_id  user_income
0        16074        53478
1        16117       465589
2        16181       134978
3        16308       175988
4        16339       197736
...        ...          ...
39692  1548081        81214
39693  1548083       156591
39694  1548094        77598
39695  1548100        95437
39696  1548191        95293

[39697 rows x 2 columns]


In [53]:
#将预测的收入数据填补进原有的数据集中
data = data.drop('user_income',axis=1)
data = pd.merge(data,user_data_all,how='left')

In [54]:
print(data)

             date  user_id user_country sex  age  new_user_type  \
0      2015-01-28  1050676           ES   V   31            0.0   
1      2015-01-28  1050662           ES   V   37            0.0   
2      2015-01-28  1050652           ES   H   31            0.0   
3      2015-01-28  1050489           ES   H   23            0.0   
4      2015-01-28  1050490           ES   V   23            0.0   
...           ...      ...          ...  ..  ...            ...   
561781 2016-05-28  1166847           ES   V   23            0.0   
561782 2016-05-28  1166605           ES   V   27            0.0   
561783 2016-05-28  1166829           ES   V   23            0.0   
561784 2016-05-28  1166827           ES   H   23            0.0   
561785 2016-05-28  1166781           ES   H   24            0.0   

       user_seniority  user_type  user_type_at_begin user_relation_at_begin  \
0                  35        1.0                 1.0                      A   
1                  35        1.0     

In [55]:
train_data_new = data[data.origin == 'train']
test_data_new = data[data.origin == 'test']
train_data_new.to_csv('train_update.csv',index = 0)
test_data_new.to_csv('test_update.csv',index = 0)